In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
random_seed = 0
save_to_disk = True
model_prefix = 'nn'

In [ ]:
from numpy.random import seed
seed(random_seed)
from tensorflow.random import set_seed
set_seed(random_seed)

In [ ]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from typing import List
import glob


In [ ]:
from knowledge_distillation.io import *
from knowledge_distillation.ensemble import UnbiasedAverage
from knowledge_distillation.processing import * 
from knowledge_distillation.nn import *

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Load data

In [ ]:
df = load_adult()

# Preprocessing

In [ ]:
X, y, target_names = scale_onehot(df, target='salary')

X_train, X_test, y_train, y_test = split_with_seed(X, y)

/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif

# Load models

In [ ]:
model_paths = sorted(glob.glob(f"{ASSETS_PATH / model_prefix}_*.tf"))

In [ ]:
model_names = [p.split('/')[-1].split('.')[0] for p in model_paths]
model_names

['nn_1', 'nn_2']

In [ ]:
# load models
models = [load_keras_classifier(name) for name in model_names]    

In [ ]:
first = models[0]
assert len(first.predict(X_test.head(5))) == 5

/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
first.predict_proba(X_test.head(5))[:,1], first.predict(X_test.head(5))

/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


(array([0.0076542 , 0.2546998 , 0.25310326, 0.0216367 , 0.05762338],
       dtype=float32),
 array([0, 0, 0, 0, 0], dtype=int8))

# Evaluate ensemble predictions

In [ ]:
@dataclass
class TrainedKerasEnsemble():
    keras_estimators:List[KerasClassifier]
    final_estimator:BaseEstimator
    
    def predict_proba(self, X):
        individual_preds = [model.predict_proba(X) for model in self.keras_estimators]
        individual_preds = np.stack(individual_preds, axis=1)
        
        return self.final_estimator.predict_proba(individual_preds)
    
    def predict(self, X):
        return (self.predict_proba(X) > .5).astype(int)
    
    

In [ ]:
ensemble = TrainedKerasEnsemble(models, UnbiasedAverage())

In [ ]:
ensemble.predict_proba(X_train.head(5))

/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


array([[0.910709  , 0.08929093],
       [0.42169142, 0.5783086 ],
       [0.9630044 , 0.03699556],
       [0.9044428 , 0.09555721],
       [0.9769597 , 0.02304029]], dtype=float32)

In [ ]:
evaluate_model(X_train, X_test, y_train, y_test, ensemble, f"{model_prefix}_ensemble", save_to_disk=save_to_disk, target_names=target_names)


/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/tensorflow/python/

=== Train ===
              precision    recall  f1-score   support

       <=50K       0.84      0.98      0.90     19778
        >50K       0.86      0.40      0.54      6270

    accuracy                           0.84     26048
   macro avg       0.85      0.69      0.72     26048
weighted avg       0.84      0.84      0.82     26048


=== Test ===
              precision    recall  f1-score   support

       <=50K       0.84      0.98      0.90      4942
        >50K       0.86      0.40      0.55      1571

    accuracy                           0.84      6513
   macro avg       0.85      0.69      0.73      6513
weighted avg       0.84      0.84      0.82      6513




,model_name,data,accuracy,precision,recall,f1,auc
0,nn_ensemble,train,0.839220,0.856019,0.399203,0.544486,0.688958
1,nn_ensemble,test,0.840626,0.864569,0.402292,0.549088,0.691130


# Model distillation: train on ensemble output

In [ ]:
# Create a single NN, identical to the ones in the ensemble
model = KerasClassifier(
    build_fn=create_nn,
    **train_params,
    verbose=1
)


In [ ]:
y_train_pred_ensemble = ensemble.predict_proba(X_train)
y_train_pred_ensemble[:5]

/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


array([[0.910709  , 0.08929093],
       [0.42169142, 0.5783086 ],
       [0.9630044 , 0.03699556],
       [0.9044428 , 0.09555721],
       [0.9769597 , 0.02304029]], dtype=float32)

In [ ]:
# train the NN on the output of the ensemble
model.fit(X_train, y_train_pred_ensemble)


Epoch 1/50
163/163 [==============================] - 1s 3ms/step - loss: 2.3592 - accuracy: 0.8805 - val_loss: 0.5102 - val_accuracy: 0.8862
Epoch 2/50
163/163 [==============================] - 0s 2ms/step - loss: 0.3885 - accuracy: 0.9428 - val_loss: 0.4433 - val_accuracy: 0.8912
Epoch 3/50
163/163 [==============================] - 0s 2ms/step - loss: 0.3809 - accuracy: 0.9462 - val_loss: 0.3854 - val_accuracy: 0.9109
Epoch 4/50
163/163 [==============================] - 0s 2ms/step - loss: 0.3707 - accuracy: 0.9532 - val_loss: 0.3764 - val_accuracy: 0.9347
Epoch 5/50
163/163 [==============================] - 0s 2ms/step - loss: 0.3645 - accuracy: 0.9527 - val_loss: 0.4108 - val_accuracy: 0.8940
Epoch 6/50
163/163 [==============================] - 0s 2ms/step - loss: 0.3651 - accuracy: 0.9550 - val_loss: 0.3905 - val_accuracy: 0.9042
Epoch 7/50
163/163 [==============================] - 0s 2ms/step - loss: 0.3603 - accuracy: 0.9556 - val_loss: 0.3794 - val_accuracy: 0.9081
Epoch 

# Evaluate distilled model

In [ ]:
evaluate_model(X_train, X_test, y_train, y_test, model, f'{model_prefix}_{random_seed}_distilled', save_to_disk=save_to_disk, target_names=target_names)


 91/204 [============>.................] - ETA: 0s 

/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


51/51 [==============================] - 0s 568us/step
=== Train ===
              precision    recall  f1-score   support

       <=50K       0.79      0.99      0.88     19778
        >50K       0.91      0.18      0.30      6270

    accuracy                           0.80     26048
   macro avg       0.85      0.59      0.59     26048
weighted avg       0.82      0.80      0.74     26048


=== Test ===
              precision    recall  f1-score   support

       <=50K       0.79      1.00      0.88      4942
        >50K       0.92      0.17      0.29      1571

    accuracy                           0.80      6513
   macro avg       0.86      0.58      0.59      6513
weighted avg       0.82      0.80      0.74      6513




/usr/local/anaconda3/envs/distillation/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


,model_name,data,accuracy,precision,recall,f1,auc
0,nn_0_distilled,train,0.798756,0.912520,0.181340,0.302555,0.587914
1,nn_0_distilled,test,0.797328,0.922559,0.174411,0.293362,0.584879
